In [1]:
# import libraries 
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math 
from scipy import stats

In [2]:
# import stocks and API token
stocks = pd.read_csv('..\data\sp_500_stocks.csv')

from secrets import IEX_CLOUD_API_TOKEN

In [3]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   

In [4]:
# creates chunks of 100 from the stock list and take every group of stocks in the symbol_groups to be a comma separated string
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

In [5]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# metrics for value investing
# D/E Ratio - proportion of equity to debt a company is using to finance its assets.
de_ratio = data[symbol]['advanced-stats']['debtToEquity']

# P/E Ratio - what the market is willing to pay for a stock based on past or future earnings.
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio - measures whether a stock is over/undervalued by comparing the net value (assets - liabilities) to its market capitalization.
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio - compares stock price to its revenues.
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

#PEG Ratio - measures the relationship between the price/earnings ratio and earnings growth
peg_ratio = data[symbol]['advanced-stats']['pegRatio']

# EV/EBITDA - comparing the value of a company, debt included, to the company’s cash earnings less non-cash expenses.
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP - how many dollars of enterprise value are generated for every dollar of gross profit earned.
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

In [6]:
# Create columns
my_columns = [
    'Ticker', 
    'Price', 
    'Number of Shares to Buy',
    'Price-to-Earnings-to-Growth Ratio',
    'PEG Percentile',
    'Debt-to-Equity Ratio',
    'DE Percentile',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

In [7]:
# Create an empty dataframe with the specified columns
df = pd.DataFrame(columns = my_columns)

In [8]:
# loop data from batch API call and applying it to the df
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        ev = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gp = data[symbol]['advanced-stats']['grossProfit']

        try:
            ev_to_ebitda = ev/ebitda
        except:
            ev_to_ebitda = np.NaN

        try:
            ev_to_gross_profit = ev/gp
        except:
            ev_to_gross_profit = np.NaN
        
        df = df.append(
            pd.Series(
                [ 
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['advanced-stats']['pegRatio'],
                'N/A',
                data[symbol]['advanced-stats']['debtToEquity'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
                ],
                index = my_columns
            ),
            ignore_index = True
        )

df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings-to-Growth Ratio,PEG Percentile,Debt-to-Equity Ratio,DE Percentile,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,160.20,N/A,1.096887,N/A,2.224621,N/A,49.57,N/A,10.12,N/A,7.97,N/A,32.539079,N/A,15.074749,N/A,N/A
1,AAL,18.45,N/A,-0.057593,N/A,-9.516004,N/A,-3.75,N/A,-1.67,N/A,0.497,N/A,-7.421145,N/A,1.490412,N/A,N/A
2,AAP,237.29,N/A,0.52472,N/A,3.791231,N/A,24.14,N/A,4.75,N/A,1.37,N/A,12.699908,N/A,2.950094,N/A,N/A
3,AAPL,184.86,N/A,0.111326,N/A,5.585186,N/A,16.0,N/A,23.15,N/A,3.96,N/A,12.017156,N/A,9.799276,N/A,N/A
4,ABBV,133.46,N/A,-3.765605,N/A,11.507808,N/A,32.12,N/A,18.08,N/A,4.32,N/A,11.599328,N/A,8.299562,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,135.89,N/A,0.518498,N/A,-0.825391,N/A,26.45,N/A,-5.25,N/A,6.27,N/A,22.614347,N/A,10.454406,N/A,N/A
501,ZBH,127.91,N/A,0.051418,N/A,1.979854,N/A,33.85,N/A,2.11,N/A,3.41,N/A,18.889075,N/A,6.020319,N/A,N/A
502,ZBRA,586.18,N/A,0.479209,N/A,2.124269,N/A,38.94,N/A,11.37,N/A,5.97,N/A,29.185898,N/A,12.715739,N/A,N/A
503,ZION,63.59,N/A,0.016831,N/A,11.763934,N/A,6.05,N/A,1.33,N/A,2.9,N/A,4.592190,N/A,2.555341,N/A,N/A


In [9]:
# Remove None
df.dropna(inplace = True)
df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings-to-Growth Ratio,PEG Percentile,Debt-to-Equity Ratio,DE Percentile,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,160.20,N/A,1.096887,N/A,2.224621,N/A,49.57,N/A,10.12,N/A,7.97,N/A,32.539079,N/A,15.074749,N/A,N/A
1,AAL,18.45,N/A,-0.057593,N/A,-9.516004,N/A,-3.75,N/A,-1.67,N/A,0.497,N/A,-7.421145,N/A,1.490412,N/A,N/A
2,AAP,237.29,N/A,0.52472,N/A,3.791231,N/A,24.14,N/A,4.75,N/A,1.37,N/A,12.699908,N/A,2.950094,N/A,N/A
3,AAPL,184.86,N/A,0.111326,N/A,5.585186,N/A,16.0,N/A,23.15,N/A,3.96,N/A,12.017156,N/A,9.799276,N/A,N/A
4,ABBV,133.46,N/A,-3.765605,N/A,11.507808,N/A,32.12,N/A,18.08,N/A,4.32,N/A,11.599328,N/A,8.299562,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,135.89,N/A,0.518498,N/A,-0.825391,N/A,26.45,N/A,-5.25,N/A,6.27,N/A,22.614347,N/A,10.454406,N/A,N/A
501,ZBH,127.91,N/A,0.051418,N/A,1.979854,N/A,33.85,N/A,2.11,N/A,3.41,N/A,18.889075,N/A,6.020319,N/A,N/A
502,ZBRA,586.18,N/A,0.479209,N/A,2.124269,N/A,38.94,N/A,11.37,N/A,5.97,N/A,29.185898,N/A,12.715739,N/A,N/A
503,ZION,63.59,N/A,0.016831,N/A,11.763934,N/A,6.05,N/A,1.33,N/A,2.9,N/A,4.592190,N/A,2.555341,N/A,N/A


In [10]:
from scipy.stats import percentileofscore as score

# Calculating value score percentiles for each stock
metrics = {
    'Price-to-Earnings-to-Growth Ratio': 'PEG Percentile',
    'Debt-to-Equity Ratio': 'DE Percentile',
    'Price-to-Earnings Ratio': 'PE Percentile',
    'Price-to-Book Ratio': 'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in df.index:
        df.loc[row, metrics[metric]] = score(df[metric], df.loc[row, metric])/100

df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings-to-Growth Ratio,PEG Percentile,Debt-to-Equity Ratio,DE Percentile,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,160.20,N/A,1.096887,0.792608,2.224621,0.338809,49.57,0.835729,10.12,0.805955,7.97,0.819302,32.539079,0.895277,15.074749,0.850103,N/A
1,AAL,18.45,N/A,-0.057593,0.205339,-9.516004,0.036961,-3.75,0.053388,-1.67,0.055441,0.497,0.051335,-7.421145,0.024641,1.490412,0.069815,N/A
2,AAP,237.29,N/A,0.52472,0.648871,3.791231,0.681725,24.14,0.505133,4.75,0.61807,1.37,0.203285,12.699908,0.435318,2.950094,0.170431,N/A
3,AAPL,184.86,N/A,0.111326,0.414784,5.585186,0.817248,16.0,0.308008,23.15,0.936345,3.96,0.600616,12.017156,0.400411,9.799276,0.659138,N/A
4,ABBV,133.46,N/A,-3.765605,0.039014,11.507808,0.924025,32.12,0.654004,18.08,0.909651,4.32,0.634497,11.599328,0.377823,8.299562,0.581109,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,135.89,N/A,0.518498,0.646817,-0.825391,0.053388,26.45,0.562628,-5.25,0.051335,6.27,0.753593,22.614347,0.73922,10.454406,0.687885,N/A
501,ZBH,127.91,N/A,0.051418,0.340862,1.979854,0.240246,33.85,0.681725,2.11,0.309035,3.41,0.533881,18.889075,0.648871,6.020319,0.414784,N/A
502,ZBRA,586.18,N/A,0.479209,0.628337,2.124269,0.301848,38.94,0.749487,11.37,0.825462,5.97,0.738193,29.185898,0.848049,12.715739,0.774127,N/A
503,ZION,63.59,N/A,0.016831,0.256674,11.763934,0.930185,6.05,0.094456,1.33,0.147844,2.9,0.464066,4.592190,0.065708,2.555341,0.13347,N/A


In [11]:
# Calculating the robust value score
# RV score is the arithmetic mean of all the percentile scores

from statistics import mean 

for row in df.index:
    value_percentiles = []

    for metric in metrics.keys():
        value_percentiles.append(df.loc[row, metrics[metric]])
    df.loc[row, 'RV Score'] = mean(value_percentiles)

df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings-to-Growth Ratio,PEG Percentile,Debt-to-Equity Ratio,DE Percentile,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,160.20,N/A,1.096887,0.792608,2.224621,0.338809,49.57,0.835729,10.12,0.805955,7.97,0.819302,32.539079,0.895277,15.074749,0.850103,0.76254
1,AAL,18.45,N/A,-0.057593,0.205339,-9.516004,0.036961,-3.75,0.053388,-1.67,0.055441,0.497,0.051335,-7.421145,0.024641,1.490412,0.069815,0.070989
2,AAP,237.29,N/A,0.52472,0.648871,3.791231,0.681725,24.14,0.505133,4.75,0.61807,1.37,0.203285,12.699908,0.435318,2.950094,0.170431,0.466119
3,AAPL,184.86,N/A,0.111326,0.414784,5.585186,0.817248,16.0,0.308008,23.15,0.936345,3.96,0.600616,12.017156,0.400411,9.799276,0.659138,0.590936
4,ABBV,133.46,N/A,-3.765605,0.039014,11.507808,0.924025,32.12,0.654004,18.08,0.909651,4.32,0.634497,11.599328,0.377823,8.299562,0.581109,0.588589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,135.89,N/A,0.518498,0.646817,-0.825391,0.053388,26.45,0.562628,-5.25,0.051335,6.27,0.753593,22.614347,0.73922,10.454406,0.687885,0.499267
501,ZBH,127.91,N/A,0.051418,0.340862,1.979854,0.240246,33.85,0.681725,2.11,0.309035,3.41,0.533881,18.889075,0.648871,6.020319,0.414784,0.452772
502,ZBRA,586.18,N/A,0.479209,0.628337,2.124269,0.301848,38.94,0.749487,11.37,0.825462,5.97,0.738193,29.185898,0.848049,12.715739,0.774127,0.695072
503,ZION,63.59,N/A,0.016831,0.256674,11.763934,0.930185,6.05,0.094456,1.33,0.147844,2.9,0.464066,4.592190,0.065708,2.555341,0.13347,0.298915


In [12]:
# Top 100 best value stocks by RV Score
df.sort_values('RV Score', ascending = True, inplace = True)
df = df[:100]
df.reset_index(drop = True, inplace = True)
df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings-to-Growth Ratio,PEG Percentile,Debt-to-Equity Ratio,DE Percentile,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,18.45,N/A,-0.057593,0.205339,-9.516004,0.036961,-3.75,0.053388,-1.67,0.055441,0.497,0.051335,-7.421145,0.024641,1.490412,0.069815,0.070989
1,HPQ,38.66,N/A,0.009867,0.232033,-23.7,0.022587,3.62,0.069815,-12.87,0.036961,0.33,0.016427,3.951240,0.051335,1.700988,0.088296,0.073922
2,MCK,247.07,N/A,0.023225,0.277207,-111.659127,0.004107,-8.09,0.049281,-64.95,0.022587,0.1488,0.008214,9.023809,0.262834,3.143164,0.180698,0.11499
3,DHI,105.72,N/A,0.02996,0.289528,1.666324,0.135524,4.6,0.078029,1.28,0.140657,0.6913,0.073922,3.609194,0.039014,2.419650,0.12731,0.126283
4,BEN,34.67,N/A,0.01957,0.264887,2.170303,0.318275,4.83,0.080082,0.7817,0.080082,1.0427,0.13963,3.046979,0.032854,0.840904,0.020534,0.133764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,PNW,71.37,N/A,-1.371082,0.10883,3.63342,0.652977,13.86,0.264887,1.33,0.147844,2.08,0.351129,10.141696,0.314168,5.622464,0.383984,0.317688
96,PGR,105.11,N/A,-0.7697,0.151951,4.026726,0.712526,15.2,0.296715,3.32,0.488706,1.27,0.180698,10.777711,0.344969,1.244085,0.055441,0.318715
97,COO,413.64,N/A,0.001724,0.213552,1.388795,0.082136,3.56,0.067762,1.61,0.212526,3.82,0.579055,19.168418,0.661191,6.221717,0.431211,0.321062
98,LOW,258.34,N/A,0.336182,0.570842,-32.163922,0.016427,22.13,0.445585,-110.21,0.014374,1.78,0.295688,14.291862,0.49692,5.999078,0.410678,0.321502


In [13]:
# Calculate the number of shares to buy
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the value of your portfolio:')

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That is not a number! \n Try again:")
        portfolio_size = input('Enter the value of your portfolio:')

In [14]:
portfolio_input()

In [15]:
# Calculates the inputted portfolio amount and equal weight on all stocks by price
position_size = float(portfolio_size)/len(df.index)

for row in df.index:
    df.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/df.loc[row, 'Price'])

df

C:\Users\dopey\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings-to-Growth Ratio,PEG Percentile,Debt-to-Equity Ratio,DE Percentile,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,18.45,5420,-0.057593,0.205339,-9.516004,0.036961,-3.75,0.053388,-1.67,0.055441,0.497,0.051335,-7.421145,0.024641,1.490412,0.069815,0.070989
1,HPQ,38.66,2586,0.009867,0.232033,-23.7,0.022587,3.62,0.069815,-12.87,0.036961,0.33,0.016427,3.951240,0.051335,1.700988,0.088296,0.073922
2,MCK,247.07,404,0.023225,0.277207,-111.659127,0.004107,-8.09,0.049281,-64.95,0.022587,0.1488,0.008214,9.023809,0.262834,3.143164,0.180698,0.11499
3,DHI,105.72,945,0.02996,0.289528,1.666324,0.135524,4.6,0.078029,1.28,0.140657,0.6913,0.073922,3.609194,0.039014,2.419650,0.12731,0.126283
4,BEN,34.67,2884,0.01957,0.264887,2.170303,0.318275,4.83,0.080082,0.7817,0.080082,1.0427,0.13963,3.046979,0.032854,0.840904,0.020534,0.133764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,PNW,71.37,1401,-1.371082,0.10883,3.63342,0.652977,13.86,0.264887,1.33,0.147844,2.08,0.351129,10.141696,0.314168,5.622464,0.383984,0.317688
96,PGR,105.11,951,-0.7697,0.151951,4.026726,0.712526,15.2,0.296715,3.32,0.488706,1.27,0.180698,10.777711,0.344969,1.244085,0.055441,0.318715
97,COO,413.64,241,0.001724,0.213552,1.388795,0.082136,3.56,0.067762,1.61,0.212526,3.82,0.579055,19.168418,0.661191,6.221717,0.431211,0.321062
98,LOW,258.34,387,0.336182,0.570842,-32.163922,0.016427,22.13,0.445585,-110.21,0.014374,1.78,0.295688,14.291862,0.49692,5.999078,0.410678,0.321502


In [16]:

# xlsxwriter library to create excel file
writer = pd.ExcelWriter('100_value_stock_strategy.xlsx', engine = 'xlsxwriter')

df.to_excel(writer, sheet_name = '100 Value Stock Strategy', index = False)

In [17]:
# excel format
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
# apply formats to columns
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['Price-to-Earnings-to-Growth Ratio', float_template],
    'E': ['PEG Percentile', percent_template],
    'F': ['Debt-to-Equity Ratio', float_template],
    'G': ['DE Percentile', percent_template],
    'H': ['Price-to-Earnings Ratio', float_template],
    'I': ['PE Percentile', percent_template],
    'J': ['Price-to-Book Ratio', float_template],
    'K': ['PB Percentile', percent_template],
    'L': ['Price-to-Sales Ratio', float_template],
    'M': ['PS Percentile', percent_template],
    'N': ['EV/EBITDA', float_template],
    'O': ['EV/EBITDA Percentile', percent_template],
    'P': ['EV/GP', float_template],
    'Q': ['EV/GP Percentile', percent_template],
    'R': ['RV Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['100 Value Stock Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['100 Value Stock Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [19]:
# save excel output
writer.save()